<a href="https://colab.research.google.com/github/stefkong1982/demo/blob/main/%D0%9F%D1%80%D0%B0%D0%BA%D1%82%D0%B8%D0%BA%D0%B0_%D0%9F%D0%BE%D0%BD%D1%8F%D1%82%D0%B8%D0%B5_%D0%BA%D0%BB%D0%B0%D1%81%D1%81%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практика

In [ ]:
from analytics_libs import AnyName

In [ ]:
from analytics_libs import *  # никогда так не делайте

In [ ]:
def cost_with_vat(cost):
    # очень полезная функция +

In [ ]:
def useful_func(a, b):
    """Функция внутри класса - НЕ использует self"""
    return a**2 + b**2

In [ ]:
class AnyName:
    def __init__(self, name):
        print('Выполняется при обращении к классу')
        # создаем все необходимые для работы класса атрибуты
        self.x = 0
        self.name = name
        self.price = 100
        self.discount = 0.1

        # приватные атрибуты
        self._api_version = '5.131'  # намек, что менять атрибут ВНЕ кода класса нехорошо
        self.__secret_attr = '5.0'  # запрет на изменение атрибута вне кода класса

    # функции внутри классов называются методами
    def method_1(self):
        # x = 1  # x это просто переменная
        self.x = 1  # x это атрибут

    def method_2(self):
        print(self.x, self.__secret_attr)

    @staticmethod  # ДЕКОРАТОР статистический метод
    def useful_func(a, b):
        """Функция внутри класса - НЕ использует self"""
        return a**2 + b**2

    @property  # вычислимый атрибут
    def price_with_discount_vat(self):
        price_discount = self.price * (1 - self.discount)
        return price_discount * 1.2

In [ ]:
a = AnyName('Ваня')

Выполняется при обращении к классу


In [ ]:
a.price_with_discount_vat

108.0

In [ ]:
a.useful_func(3, 4)

25

In [ ]:
AnyName('Ваня').useful_func(3, 4)

Выполняется при обращении к классу


25

In [ ]:
a.method_2()

0 5.0


In [ ]:
a.__secret_attr

AttributeError: 'AnyName' object has no attribute '__secret_attr'

In [ ]:
a._api_version = '6.0'
a._api_version

'6.0'

In [ ]:
a.api_version = '6.0'

In [ ]:
a.x = 5
a.x

5

In [ ]:
vasya = AnyName(name='Василий')

lena = AnyName('Елена')

Выполняется при обращении к классу
Выполняется при обращении к классу


In [ ]:
vasya + lena

In [ ]:
type(lena)

__main__.AnyName

In [ ]:
l_1 = [1, 2, 3]
l_2 = [3, 4, 5]

In [ ]:
l_1 + l_2

[1, 2, 3, 3, 4, 5]

In [ ]:
1 + 1

2

In [ ]:
__add__()

In [ ]:
'1' + '1'

'11'

In [ ]:
type(l_1)

list

In [ ]:
a.x

0

In [ ]:
a.method_2()

0


In [ ]:
a.x = 5
a.x

5

In [ ]:
a.method_2()

5


In [ ]:
def func_1():
    x = 1
    return x

def func_2(x):
    print(x)

In [ ]:
result = func_1()

In [ ]:
func_2(result)

1


# Требования к отчету
(в хронологическом порядке)

1. Нужен отчет по трафику (визиты по дням)
2. Сделайте таблицу попроще
3. А можно сразу очень много строк получить?

[Демо-счетчик](https://metrika.yandex.ru/stat/traffic?sort=-ym%3As%3Avisits&period=2020-08-01%3A2020-09-06&accuracy=1&id=44147844&stateHash=5d3ee51607d6bde783131e11) Яндекс.Метрики


In [ ]:
import requests

/Users/kbashevoy/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [ ]:
API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
METRIKA_ROWS_LIMIT = 5  # max 100000

DEMO_COUNTER = 44147844

In [ ]:
class Metrika:
    def __init__(self, token='', counter=DEMO_COUNTER, start_date='2020-09-01', end_date='2020-09-10'):
        self.counter = counter
        self.limit = METRIKA_ROWS_LIMIT
        self.start_date = start_date
        self.end_date = end_date

        if token:
            self.headers = {'Authorization': 'OAuth ' + token}
        else:
            self.headers = ''  # для демо-счетчика заголовок запроса должен быть пустым

        self.dimensions = ''  # строки отчета
        self.metrics = ''  # столбцы отчета
        self.report = []  # сам отчет

    def api_request(self, offset=1):
        """Запрос к API Метрики, возвращает словарь с отчетом"""
        params = {
            'id': self.counter,
            'date1': self.start_date,
            'date2': self.end_date,
            'metrics': self.metrics,
            'dimensions': self.dimensions,
            'limit': self.limit,
            'offset': offset,
            'accuracy': 1,
        }

        response = requests.get(API_URL, params=params, headers=self.headers)
        return response.json()['data']

    def traffic(self):
        """Отчет по посещаемости"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date'

        report = self.full_report()
        self.report = report

    def browsers(self):
        """Отчет по браузерам"""
        self.metrics = 'ym:s:visits'
        self.dimensions = 'ym:s:date,ym:s:browser'

        report = self.full_report()
        self.report = report

    def reformat_api_report(self):
        """Упрощение отчета до списка списков"""
        reformatted_report = []

        for line in self.report:
            dimensions = [x['name'] for x in line['dimensions']]
            reformatted_report.append(dimensions + line['metrics'])

        self.report = reformatted_report

    def full_report(self):
        """Постраничная выгрузка из Метрики"""
        full_data = []
        offset = 1

        while True:
            print('Starting offset {}'.format(offset))
            data = self.api_request(offset=offset)
            full_data += data

            offset += self.limit
            if not data or len(data) < self.limit:
                break

        return full_data

In [ ]:
m = Metrika(start_date='2023-08-01', end_date='2023-08-21')

In [ ]:
m.traffic()

Starting offset 1
Starting offset 6
Starting offset 11
Starting offset 16
Starting offset 21


In [ ]:
m.reformat_api_report()
m.report

[['2023-08-03', 1099.0],
 ['2023-08-04', 1099.0],
 ['2023-08-08', 1037.0],
 ['2023-08-10', 996.0],
 ['2023-08-09', 991.0],
 ['2023-08-07', 924.0],
 ['2023-08-02', 854.0],
 ['2023-08-11', 669.0],
 ['2023-08-16', 646.0],
 ['2023-08-21', 549.0],
 ['2023-08-17', 538.0],
 ['2023-08-15', 520.0],
 ['2023-08-14', 502.0],
 ['2023-08-18', 493.0],
 ['2023-08-01', 490.0],
 ['2023-08-06', 483.0],
 ['2023-08-05', 476.0],
 ['2023-08-12', 448.0],
 ['2023-08-13', 435.0],
 ['2023-08-20', 381.0],
 ['2023-08-19', 371.0]]

In [ ]:
m.browsers()
m.reformat_api_report()
m.report

Starting offset 1
Starting offset 6
Starting offset 11
Starting offset 16
Starting offset 21
Starting offset 26
Starting offset 31
Starting offset 36
Starting offset 41
Starting offset 46
Starting offset 51
Starting offset 56
Starting offset 61
Starting offset 66
Starting offset 71
Starting offset 76
Starting offset 81
Starting offset 86
Starting offset 91
Starting offset 96
Starting offset 101
Starting offset 106
Starting offset 111
Starting offset 116
Starting offset 121
Starting offset 126
Starting offset 131
Starting offset 136
Starting offset 141
Starting offset 146
Starting offset 151
Starting offset 156
Starting offset 161
Starting offset 166
Starting offset 171
Starting offset 176
Starting offset 181
Starting offset 186
Starting offset 191
Starting offset 196
Starting offset 201
Starting offset 206
Starting offset 211
Starting offset 216
Starting offset 221
Starting offset 226
Starting offset 231
Starting offset 236
Starting offset 241
Starting offset 246
Starting offset 251
St

[['2023-08-03', 'Google Chrome', 458.0],
 ['2023-08-04', 'Google Chrome', 443.0],
 ['2023-08-08', 'Google Chrome', 407.0],
 ['2023-08-09', 'Google Chrome', 406.0],
 ['2023-08-10', 'Google Chrome', 395.0],
 ['2023-08-07', 'Google Chrome', 381.0],
 ['2023-08-02', 'Google Chrome', 334.0],
 ['2023-08-11', 'Google Chrome', 263.0],
 ['2023-08-03', 'Яндекс.Браузер', 249.0],
 ['2023-08-10', 'Яндекс.Браузер', 235.0],
 ['2023-08-04', 'Яндекс.Браузер', 232.0],
 ['2023-08-16', 'Google Chrome', 226.0],
 ['2023-08-08', 'Яндекс.Браузер', 217.0],
 ['2023-08-16', 'Chrome Mobile', 214.0],
 ['2023-08-20', 'Chrome Mobile', 212.0],
 ['2023-08-17', 'Chrome Mobile', 204.0],
 ['2023-08-01', 'Google Chrome', 199.0],
 ['2023-08-21', 'Chrome Mobile', 199.0],
 ['2023-08-09', 'Яндекс.Браузер', 198.0],
 ['2023-08-07', 'Яндекс.Браузер', 192.0],
 ['2023-08-19', 'Chrome Mobile', 192.0],
 ['2023-08-08', 'Chrome Mobile', 190.0],
 ['2023-08-18', 'Chrome Mobile', 189.0],
 ['2023-08-21', 'Google Chrome', 189.0],
 ['2023-08

In [ ]:
i = 0

while i < 5:
    print(i)

    i += 1

0
1
2
3
4


In [ ]:
while True:
    # пока внутри него вручную не сработает какой-нибудь break